In [1]:
import matplotlib
import matplotlib.pyplot as plt
import os
import logging
import glob

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [2]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [3]:
%ls '/home/labuser/LogoDet/LogoDetection_DSBAProject/training_process/training/TENSOR_RESULTS'

efficientdet-d0/


In [4]:
model = "efficientdet-d0"
config = "config_7"
output_directory = f'/home/labuser/LogoDet/LogoDetection_DSBAProject/training_process/training/FINE_TUNED_MODEL/{model}/{config}'

In [5]:
import pathlib

filenames = list(pathlib.Path(os.path.join(output_directory,"checkpoint")).glob('*.index'))

filenames.sort()
print(filenames)

#recover our saved model
pipeline_file = f"/home/labuser/LogoDet/LogoDetection_DSBAProject/training_process/training/models/research/deploy/{model}/config_7/pipeline_file.config"
pipeline_config = pipeline_file

#generally you want to put the last ckpt from training in here
model_dir = str(filenames[-1]).replace('.index','')
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(str(filenames[-1]).replace('.index','')))


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)
    
    
    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

[PosixPath('/home/labuser/LogoDet/LogoDetection_DSBAProject/training_process/training/FINE_TUNED_MODEL/efficientdet-d0/config_7/checkpoint/ckpt-0.index')]


In [6]:
#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [7]:
%cd /home/labuser/LogoDet/LogoDetection_DSBAProject/inference

/home/labuser/LogoDet/LogoDetection_DSBAProject/inference


In [8]:
if "INFERENCE_DIR" not in os.listdir(os.getcwd()):
    os.mkdir(output_directory)
    logging.info("Creating the directory INFERENCE_DIRbecause it did not exist") 
else:
    logging.info("The directory INFERENCE_DIR is already present, files will be stored there") 

In [9]:
%cd /home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR

/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR


In [12]:
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed. 
import random
from build_csv import build_predicion_CSV

TEST_IMAGE_PATHS = glob.glob('/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR_v2/*.jpg')

counter = 0
for image in TEST_IMAGE_PATHS[:500]:
    image_path = image
    print(image)
    #image_path = "/home/labuser/LogoDet/LogoDetection_DSBAProject/training_process/training/INFERENCE_DIR/NFLogo1.jpg"
    image_np = load_image_into_numpy_array(image_path)

    input_tensor = tf.convert_to_tensor(
        np.expand_dims(image_np, 0), dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    threshold = 0.4
    
#    viz_utils.visualize_boxes_and_labels_on_image_array(
#                  image_np_with_detections,
#                  detections['detection_boxes'][0].numpy(),
#                  (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
#                  detections['detection_scores'][0].numpy(),
#                  category_index,
#                  use_normalized_coordinates=True, # Needs to be true cause otherwise boxes go out of range
#                  max_boxes_to_draw=3, # Minimized to 10 because otherwise we might have troubles 
#                  min_score_thresh=threshold, # Minimized the score, as for now it seems to be too confident over Adidas
#                  agnostic_mode=False)
    
    build_predicion_CSV(threshold=threshold, name_image=image,counter = counter, detections= detections,category=category_index)
    counter = counter + 1

#    plt.figure(figsize=(12,16))
#    plt.imshow(image_np_with_detections)
#    plt.show()
    

/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR_v2/la_1357350732949111450_20161009_jpg.rf.88234f398f606568c296e9dd1204072c.jpg
{'/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR_v2/la_1357350732949111450_20161009_jpg.rf.88234f398f606568c296e9dd1204072c.jpg': {0: {'label': 8, 'accuracy': 0.5136926, 'prediction': [0.6787775, 0.6797474, 0.8099101, 0.85657126]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR_v2/london_1836721383621492519_20180802_jpg.rf.ffaf084b79138435fa466b830bbc9751.jpg
{'/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR_v2/london_1836721383621492519_20180802_jpg.rf.ffaf084b79138435fa466b830bbc9751.jpg': {0: {'label': 8, 'accuracy': 0.5152991, 'prediction': [0.8625742, 0.8827025, 0.968433, 0.9801765]}}}
/home/labuser/LogoDet/LogoDetection_DSBAProject/inference/INFERENCE_DIR_v2/la_2017_1590040429369621714_20170826_jpg.rf.7f5ec78f52535d4f7f18888a4a90353c.jpg
{'/home/labuser/Logo

KeyboardInterrupt: 